In [1]:
# for data analysis
import pandas as pd
import numpy as np

# unsupervised learning model for clustering
# customers into new segements
from sklearn.cluster import KMeans

# constant for random state
RANDOM_STATE = 42

# Project content

1. DATA MANIPULATION TASK
2. DATA VISUALISATION TASK
3. MODELLING TASK

In [2]:
# load data
df_bank = pd.read_csv('data/bank.csv', delimiter=';')
df_bank.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [3]:
# load data
df_bank_full = pd.read_csv('data/bank-full.csv', delimiter=';')
df_bank_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [4]:
# read .txt file
with open('data/bank-names.txt', 'r') as reader:
    print(reader.read())

Citation Request:
  This dataset is public available for research. The details are described in [Moro et al., 2011]. 
  Please include this citation if you plan to use this database:

  [Moro et al., 2011] S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European Simulation and Modelling Conference - ESM'2011, pp. 117-121, GuimarÃ£es, Portugal, October, 2011. EUROSIS.

  Available at: [pdf] http://hdl.handle.net/1822/14838
                [bib] http://www3.dsi.uminho.pt/pcortez/bib/2011-esm-1.txt

1. Title: Bank Marketing

2. Sources
   Created by: Paulo Cortez (Univ. Minho) and SÃ©rgio Moro (ISCTE-IUL) @ 2012
   
3. Past Usage:

  The full dataset was described and analyzed in:

  S. Moro, R. Laureano and P. Cortez. Using Data Mining for Bank Direct Marketing: An Application of the CRISP-DM Methodology. 
  In P. Novais et al. (Eds.), Proceedings of the European

In [5]:
# reduce memory
# reduce memory usage if available
def reduce_mem_usage(df, df_name, verbose=True):
    """
    This function changes pandas numerical dtypes (reduces bit size if possible)
    to reduce memory usage
    
    :param df: pandas DataFrame
    :param df_name: str, name of DataFrame
    :param verbose: bool, if True prints out message of how much memory usage was reduced
        
    :return:  pandas DataFrame       
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int64)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float32)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float64)
                else:
                    df[col] = df[col].astype(np.float64)
    # calculate memory after reduction
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        # reduced memory usage in percent
        diff_pst = 100 * (start_mem - end_mem) / start_mem
        msg = f'{df_name} mem. usage decreased to {end_mem:5.2f} Mb ({diff_pst:.1f}% reduction)'
        print(msg)
    return df

In [6]:
df_bank_full = reduce_mem_usage(df_bank_full, "df_bank_full")
df_bank = reduce_mem_usage(df_bank, "df_bank")

df_bank_full mem. usage decreased to  4.57 Mb (22.1% reduction)
df_bank mem. usage decreased to  0.45 Mb (23.5% reduction)


## 1.  DATA MANIPULATION TASK

In [7]:
print(f'df_bank_full DataFrame contains: {len(df_bank_full)} rows.')
print(f'df_bank      DataFrame contains: {len(df_bank)} rows.')

df_bank_full DataFrame contains: 45211 rows.
df_bank      DataFrame contains: 4521 rows.


* **Task 1** select random subsample of data set

In [8]:
# select N random subsample of data set
N = 100
# random state parameter is used for reproducibility
df_sample = df_bank_full.sample(N, random_state=RANDOM_STATE)
# return 5 first rows of random sample
df_sample.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
3776,40,blue-collar,married,secondary,no,580,yes,no,unknown,16,may,192,1,-1,0,unknown,no
9928,47,services,single,secondary,no,3644,no,no,unknown,9,jun,83,2,-1,0,unknown,no
33409,25,student,single,tertiary,no,538,yes,no,cellular,20,apr,226,1,-1,0,unknown,no
31885,42,management,married,tertiary,no,1773,no,no,cellular,9,apr,311,1,336,1,failure,no
15738,56,management,married,tertiary,no,217,no,yes,cellular,21,jul,121,2,-1,0,unknown,no


* **Task 2** filter desired rows using simple and more complex conditions

In [9]:
# select all data for august month
df_aug = df_bank_full.loc[df_bank_full.month == 'aug']
# return first 5 rows
df_aug.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
18823,58,admin.,married,secondary,no,6878,no,no,cellular,4,aug,148,1,-1,0,unknown,no
18824,51,technician,divorced,secondary,no,1063,no,yes,cellular,4,aug,78,1,-1,0,unknown,no
18825,56,services,married,tertiary,no,3480,no,no,cellular,4,aug,124,1,-1,0,unknown,no
18826,47,management,married,tertiary,no,0,no,no,cellular,4,aug,491,1,-1,0,unknown,no
18827,32,management,married,tertiary,no,1570,no,no,cellular,4,aug,156,1,-1,0,unknown,no


In [10]:
# generate 3 conditions

# select only students, retired and unemployed data entries
cond_1 = df_bank_full.job.isin(['student', 'retired', 'unemployed'])
# select contact communication type only "cellular" 
cond_2 = df_bank_full.contact == 'cellular'
# average yearly balance (in euros) is at least 1000 eur.
cond_3 = df_bank_full.balance >= 1000

# new DataFrame with 3 conditions
df_cond = df_bank_full.loc[cond_1 & cond_2 & cond_3]
# return first 5 rows
df_cond.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
12818,51,retired,married,secondary,no,3003,yes,no,cellular,7,jul,149,3,-1,0,unknown,no
12915,52,unemployed,married,tertiary,no,1023,no,no,cellular,7,jul,107,6,-1,0,unknown,no
13078,57,retired,married,tertiary,no,1029,no,yes,cellular,8,jul,203,1,-1,0,unknown,no
13143,42,unemployed,divorced,secondary,no,2699,no,no,cellular,8,jul,92,2,-1,0,unknown,no
13183,60,retired,married,secondary,no,1141,no,yes,cellular,8,jul,211,2,-1,0,unknown,no


In [11]:
# use query (SQL-like) function to select all entries with housing during July month and
# the number of contacts performed during this campaign is greater than or equal to 3
df_query = df_bank_full.query('month == "jul" & campaign >= 3')
# return first 5 rows
df_query.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
12443,59,retired,married,secondary,yes,49,no,yes,unknown,1,jul,602,4,-1,0,unknown,no
12444,41,services,married,secondary,no,138,yes,no,unknown,1,jul,136,3,-1,0,unknown,no
12445,30,blue-collar,married,secondary,no,365,no,no,unknown,1,jul,381,4,-1,0,unknown,no
12447,41,blue-collar,single,secondary,no,75,yes,no,unknown,1,jul,479,9,-1,0,unknown,yes
12448,31,blue-collar,married,secondary,no,652,no,yes,unknown,1,jul,281,3,-1,0,unknown,no


* **Task 3** drop unnecessary variables, rename some variables;

In [12]:
# delete y column from sample DataFrame
# before
df_sample.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
3776,40,blue-collar,married,secondary,no,580,yes,no,unknown,16,may,192,1,-1,0,unknown,no
9928,47,services,single,secondary,no,3644,no,no,unknown,9,jun,83,2,-1,0,unknown,no
33409,25,student,single,tertiary,no,538,yes,no,cellular,20,apr,226,1,-1,0,unknown,no
31885,42,management,married,tertiary,no,1773,no,no,cellular,9,apr,311,1,336,1,failure,no
15738,56,management,married,tertiary,no,217,no,yes,cellular,21,jul,121,2,-1,0,unknown,no


In [13]:
# after
del df_sample['y']
df_sample.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
3776,40,blue-collar,married,secondary,no,580,yes,no,unknown,16,may,192,1,-1,0,unknown
9928,47,services,single,secondary,no,3644,no,no,unknown,9,jun,83,2,-1,0,unknown
33409,25,student,single,tertiary,no,538,yes,no,cellular,20,apr,226,1,-1,0,unknown
31885,42,management,married,tertiary,no,1773,no,no,cellular,9,apr,311,1,336,1,failure
15738,56,management,married,tertiary,no,217,no,yes,cellular,21,jul,121,2,-1,0,unknown


In [14]:
# capitalize all column name first letters
df_sample.columns = [col.title() for col in df_sample.columns]
df_sample.head()

,Age,Job,Marital,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,Pdays,Previous,Poutcome
3776,40,blue-collar,married,secondary,no,580,yes,no,unknown,16,may,192,1,-1,0,unknown
9928,47,services,single,secondary,no,3644,no,no,unknown,9,jun,83,2,-1,0,unknown
33409,25,student,single,tertiary,no,538,yes,no,cellular,20,apr,226,1,-1,0,unknown
31885,42,management,married,tertiary,no,1773,no,no,cellular,9,apr,311,1,336,1,failure
15738,56,management,married,tertiary,no,217,no,yes,cellular,21,jul,121,2,-1,0,unknown


In [15]:
# rename Job column to Work
df_sample.rename(columns={'Job': 'Work'}, inplace=True)
df_sample.head()

,Age,Work,Marital,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,Pdays,Previous,Poutcome
3776,40,blue-collar,married,secondary,no,580,yes,no,unknown,16,may,192,1,-1,0,unknown
9928,47,services,single,secondary,no,3644,no,no,unknown,9,jun,83,2,-1,0,unknown
33409,25,student,single,tertiary,no,538,yes,no,cellular,20,apr,226,1,-1,0,unknown
31885,42,management,married,tertiary,no,1773,no,no,cellular,9,apr,311,1,336,1,failure
15738,56,management,married,tertiary,no,217,no,yes,cellular,21,jul,121,2,-1,0,unknown


* **Task 4** calculate summarizing statistics (for full sample and by categorical variables as well)

In [16]:
# ugly, but quick way to get basic summurizing statistics for continues numerical variables
df_bank_full.describe()

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [17]:
# count missing values per column
df_bank_full.isna().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [18]:
# create custom function for calculating summarizing statistics
def summary_stats(df, col_name):
    """
    This function prints out summarizing statistics for both numerical and categorical variables
    
    :param df: pandas DataFrame
    :param col_name: str, name of DataFrame
        
    :return:  None     
    """
    # list of numeric dtypes
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    if df[col_name].dtype in numerics:
        print(f'"{col_name}" is continous numerical variable')
        # calculate mean, median and standard deviation
        _mean = df[col_name].mean()
        _median = df[col_name].median()
        _std = df[col_name].std()
        print(f'Mean: {_mean:.1f}, Median: {_median:.1f}, STD {_std:.1f}')
        # calculate skewness and kurtosis
        _skew = df[col_name].skew()
        _kurt = df[col_name].kurtosis()
        print(f'Skewness: {_skew:.2f}, kurtosis {_kurt:.2f}')
        
        if _skew >= 1:
            print('Right (possitive) skewned distribution')
        if _skew <= -1:
            print('Left (negative) skewned distribution')    
    else:
        print(f'"{col_name}" is categorical variable')
        # count categorical values
        _values = pd.DataFrame(df_bank_full[col_name].value_counts())
        # calculate value precentange of whole dataset
        _values['pct'] = _values[col_name] / len(df_bank_full) * 100
        print(f"{len(_values)} categories found.")
        if len(_values) > 5:
            print('TOP 5 categories:')
            # get only top 5 values
            _values = _values.sort_values(by=col_name, ascending=False).head(5)
            
        for idx in _values.index:
            _cnt = _values.loc[idx, col_name]
            _pct = _values.loc[idx, 'pct']
            print(f"{idx:15s} {_cnt} values represent {_pct:.1f}% of all data.")

In [19]:
# iterate over column names
for col_name in df_bank_full.columns:
    # print out 
    summary_stats(df_bank_full, col_name)
    # add emty row after calculations
    if col_name != 'y':
        print()

"age" is continous numerical variable
Mean: 40.9, Median: 39.0, STD 10.6
Skewness: 0.68, kurtosis 0.32

"job" is categorical variable
12 categories found.
TOP 5 categories:
blue-collar     9732 values represent 21.5% of all data.
management      9458 values represent 20.9% of all data.
technician      7597 values represent 16.8% of all data.
admin.          5171 values represent 11.4% of all data.
services        4154 values represent 9.2% of all data.

"marital" is categorical variable
3 categories found.
married         27214 values represent 60.2% of all data.
single          12790 values represent 28.3% of all data.
divorced        5207 values represent 11.5% of all data.

"education" is categorical variable
4 categories found.
secondary       23202 values represent 51.3% of all data.
tertiary        13301 values represent 29.4% of all data.
primary         6851 values represent 15.2% of all data.
unknown         1857 values represent 4.1% of all data.

"default" is categorical var

* **Task 5** create new variables using simple transformation and custom functions

In [20]:
# before adding new features
df_bank_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [21]:
# create new feature from unsupervised learning
# cluster customers into 5 groups
# based on their age, balance and last contact duration
X = df_bank_full[['age', 'balance', 'duration']]
kmeans = KMeans(n_clusters=5, random_state=RANDOM_STATE).fit(X)
df_bank_full['cluster_no'] = kmeans.labels_
df_bank_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,cluster_no
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,3
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,0
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,0
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,0
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,0


In [22]:
_ = df_bank_full['cluster_no'].value_counts() / len(df_bank_full) * 100
print('Cluster reresents (% of data)')
_.round(1)

Cluster reresents (% of data)


0    80.0
3    15.4
1     3.8
4     0.7
2     0.1
Name: cluster_no, dtype: float64

In [23]:
# create new variable with date from 2 columns
df_bank_full['dummy_date'] = df_bank_full.apply(lambda x: f"{x['month']}-{x['day']}", axis=1)
df_bank_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,cluster_no,dummy_date
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,3,may-5
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,0,may-5
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,0,may-5
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,0,may-5
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,0,may-5


If year data was available, I could create new features, e.g. weekeday, holiday.
* From **[Moro et al., 2011]** paper:
    
    "*The dataset collected is related to 17 campaigns that occurred between May 2008 and November 2010, corresponding to a total of 79354 contacts.*"
    
Real date can't be extracted from provided dataset.

In [24]:
# cut customers into 5 groups based on age quantiles
ser, bins = pd.qcut(df_bank_full['age'], 5, duplicates='drop', retbins=True, labels=False)
df_bank_full['age_range'] = ser
df_bank_full.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y,cluster_no,dummy_date,age_range
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no,3,may-5,4
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no,0,may-5,3
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no,0,may-5,1
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no,0,may-5,3
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no,0,may-5,1


In [25]:
print('Age groups')
for _bin in range(len(bins) - 1):
    _range = f'({bins[_bin]} - {bins[_bin+1]}]'
    # first range includes first bin
    if not _bin: _range = '[' + _range[1:]
    # count values and percentage this age range represent of all data
    _ = (df_bank_full.age_range ==_bin)
    print(f'{_range} {_.sum()} values ({_.mean()*100:.1f}% of all data)')

Age groups
[18.0 - 32.0] 11111 values (24.6% of all data)
(32.0 - 36.0] 7602 values (16.8% of all data)
(36.0 - 42.0] 8537 values (18.9% of all data)
(42.0 - 51.0] 9642 values (21.3% of all data)
(51.0 - 95.0] 8319 values (18.4% of all data)


* **Task 6** order data set by several variables

In [26]:
# order by average yearly balance, in euros and 
# by last contact duration, in seconds
df_sample = df_sample.sort_values(by=['Balance', 'Duration'])

# get top 5 rows
df_sample.head()

,Age,Work,Marital,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,Pdays,Previous,Poutcome
34619,43,blue-collar,married,secondary,no,-755,yes,yes,cellular,5,may,162,1,337,1,failure
26191,49,self-employed,divorced,secondary,no,-478,yes,yes,cellular,20,nov,291,1,-1,0,unknown
5847,31,blue-collar,divorced,secondary,no,-452,yes,no,unknown,26,may,182,5,-1,0,unknown
38857,31,technician,single,secondary,no,-348,yes,no,cellular,18,may,410,1,-1,0,unknown
25089,45,self-employed,married,tertiary,no,-312,no,no,telephone,18,nov,46,4,-1,0,unknown


In [27]:
# get botom 5 rows
df_sample.tail()

,Age,Work,Marital,Education,Default,Balance,Housing,Loan,Contact,Day,Month,Duration,Campaign,Pdays,Previous,Poutcome
1470,60,blue-collar,divorced,primary,no,7601,yes,no,unknown,8,may,285,1,-1,0,unknown
2055,51,blue-collar,married,tertiary,no,7816,yes,no,unknown,12,may,76,2,-1,0,unknown
9604,36,services,married,secondary,no,8114,yes,no,unknown,6,jun,505,1,-1,0,unknown
26528,32,blue-collar,married,secondary,no,10142,yes,no,cellular,20,nov,249,1,-1,0,unknown
29675,33,unemployed,single,tertiary,no,14889,yes,no,cellular,3,feb,22,3,-1,0,unknown
